In [ ]:
#
# day 7 lab
#
#
# IMPROVE THE TEST ACCURACY SCORE AS MUCH AS YOU CAN USING YESTERDAY'S CODE!
#
# You can do this in several ways:
# 1) Rewrite the ML pipeline to use RandomForestClassifier instead of LogisticRegression.
#    The hyperparameters in a random forest are n_estimators and max_depth.
# 2) Improve the preprocessing by removing words with numbers and special characters in them. 
# 3) Check out the help of TfidfVectorizer and adjust some of its parameters. 
# 4) Treat the number of topics in LDA as a hyperparameter.
# 5) Be creative! :)

In [53]:
# step 1: load the 20 newsgroups dataset and other packages that you'll need

from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import re
from nltk.stem import PorterStemmer

np.random.seed(1)

newsgroups_train = fetch_20newsgroups(subset='train', 
                                      shuffle=True, 
                                      random_state=1, 
                                      remove=('headers', 'footers', 'quotes'))
corpus = newsgroups_train.data[:1000]
target = newsgroups_train.target[:1000]
#print(target)
#print(newsgroups_train.target_names)

newsgroups_test = fetch_20newsgroups(subset='test', 
                                      shuffle=True, 
                                      random_state=1, 
                                      remove=('headers', 'footers', 'quotes'))
corpus_test = newsgroups_test.data[:200]
target_test = newsgroups_test.target[:200]

# # preprocess corpus and test corpus
# def preprocess(docs):
#     docs = [doc.lower() for doc in docs]
#     docs = [re.sub(r'[^\w\s]', '', doc) for doc in docs]
#     new_docs = []
#     ps = PorterStemmer()
#     for doc in docs:
#         stemmed = [ps.stem(word) for word in doc.split()]
#         new_docs.append(' '.join(stemmed))
#     return new_docs

# corpus = preprocess(corpus)
# corpus_test = preprocess(corpus_test)

# step 2: use the code you developed yesterday to convert the corpus into a feature matrix
t2_vectorizer = TfidfVectorizer(ngram_range=(1,2), stop_words='english',min_df=5)
X = t2_vectorizer.fit_transform(corpus)
X_test = t2_vectorizer.transform(corpus_test)

# print(np.shape(t2))
# print(np.shape(t2_test))

# n_topics = 30
# LDA = LatentDirichletAllocation(n_components=n_topics, random_state=0)
# X = LDA.fit_transform(t2)
# X_test = LDA.transform(t2_test)

print(np.shape(X))
print(np.shape(X_test))

# step 3: put together a ML pipeline using sklearn.linear_model.LogisticRegression
# note: there is already a test in 20newsgroups so you only need to create train and CV sets
# your hyperparameters are the four different preprocessing steps and C
# check the LogisticRegression help to learn about C!
# use accuracy as your evaluation metric (sklearn.metrics.accuracy_score)

# step 3.1: split data into two parts (train and CV, 75% - 25%)
X_train, X_CV, target_train, target_CV = train_test_split(X,target,test_size = 0.25,train_size =0.75)
#print(np.shape(X_train))
#print(np.shape(X_CV))
#print(np.shape(target_train))
#print(np.shape(target_CV))

n_estimators = [1,3,10,30,100,300,1000]
max_depth = [1,3,10,30,100]

# arrays for training and test scores
train_score = np.zeros([len(n_estimators),len(max_depth)])
CV_score = np.zeros([len(n_estimators),len(max_depth)])

# loop through the combinations of hyperparameters
for n in range(len(n_estimators)):
    for m in range(len(max_depth)):
        # initialize the classifier
        clf = RandomForestClassifier(n_estimators = n_estimators[n],max_depth=max_depth[m],random_state=10)
        # fit the training data
        clf.fit(X_train,target_train)
        # predict the training data
        y_p_train = clf.predict(X_train)
        # predict the test data
        y_p_CV = clf.predict(X_CV)
        
        # measure classifier performance with accuracy: what fraction of points are correctly classified?
        train_score[n,m] = accuracy_score(target_train,y_p_train)
        CV_score[n,m] = accuracy_score(target_CV,y_p_CV)

# step 3.5: print the train and CV scores
print(train_score)
print(CV_score)
# check CV_score to figure out which hyperparameter value to use!


(1000, 2889)
(200, 2889)
[[0.06266667 0.07733333 0.168      0.32666667 0.58133333]
 [0.10533333 0.15866667 0.30666667 0.51333333 0.79066667]
 [0.16133333 0.28       0.49466667 0.74666667 0.95866667]
 [0.25333333 0.42       0.68133333 0.88666667 0.97066667]
 [0.25866667 0.488      0.77866667 0.93733333 0.97333333]
 [0.264      0.49066667 0.80666667 0.94933333 0.97466667]
 [0.27866667 0.49333333 0.804      0.948      0.97466667]]
[[0.052 0.06  0.104 0.2   0.26 ]
 [0.052 0.08  0.16  0.252 0.284]
 [0.072 0.18  0.3   0.396 0.388]
 [0.164 0.3   0.428 0.448 0.428]
 [0.172 0.34  0.46  0.504 0.488]
 [0.188 0.336 0.488 0.516 0.516]
 [0.216 0.348 0.468 0.52  0.5  ]]


In [54]:
# step 3.6: initialize the best classifier and report the test error
clf = RandomForestClassifier(n_estimators = 1000,max_depth=30,random_state=10)
# fit the model to  determine the best parameters
clf.fit(X_train,target_train)
# predict the test set
y_pred = clf.predict(X_test)
print(accuracy_score(target_test,y_pred))


0.465
